In [9]:
import pandas as pd

In [179]:
data_offline= pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')

In [180]:
data_online.describe()

,User_id,Merchant_id,Action
count,1.142983e+07,1.142983e+07,1.142983e+07
mean,1.074683e+07,3.436686e+04,2.348193e-01
std,4.137712e+06,1.441243e+04,5.426314e-01
min,4.000000e+00,1.000100e+04,0.000000e+00
25%,1.019827e+07,2.090100e+04,0.000000e+00
50%,1.196970e+07,3.420000e+04,0.000000e+00
75%,1.373506e+07,4.741500e+04,0.000000e+00
max,1.550000e+07,6.000000e+04,2.000000e+00


In [181]:
data_offline.shape

(1754884, 7)

In [182]:
data_online.shape

(11429826, 7)

In [183]:
data_online[278:283]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
278,10131831,23612,0,null,null,null,20160421
279,10131831,47003,1,fixed,fixed,20160117,20160117
280,10131831,47003,1,fixed,fixed,20160117,20160117
281,10131831,47003,2,100215730,20:10,20160117,null
282,10727799,33701,0,null,null,null,20160513


# (1) Online

# Number of time this customer gets a coupon online

In [184]:
a=data_online[['User_id','Coupon_id']]
mask = data_online['Coupon_id']!='null'

In [185]:
num_coupon = a[mask].groupby(['User_id']).count().reset_index()

In [186]:
num_coupon [:5]

,User_id,Coupon_id
0,4,1
1,209,1
2,215,2
3,236,3
4,387,8


In [187]:
num_coupon.columns = ['User_id','Tot_coupon_got']

In [188]:
data_online1 = pd.merge(data_online, num_coupon, on = 'User_id', how='left')

In [189]:
data_online1 [:5]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Tot_coupon_got
0,13740231,18907,2,100017492,500:50,20160513,null,6.0
1,13740231,34805,1,null,null,null,20160321,6.0
2,14336199,18907,0,null,null,null,20160618,NaN
3,14336199,18907,0,null,null,null,20160618,NaN
4,14336199,18907,0,null,null,null,20160618,NaN


# Number of time this customer uses a coupon

In [190]:
mask2 = (data_online1['Date']!= 'null') & (data_online1['Coupon_id']!= 'null')

In [191]:
data_online1[mask2][:5]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Tot_coupon_got
273,10131831,47003,1,fixed,fixed,20160128,20160128,8.0
274,10131831,47003,1,fixed,fixed,20160128,20160128,8.0
279,10131831,47003,1,fixed,fixed,20160117,20160117,8.0
280,10131831,47003,1,fixed,fixed,20160117,20160117,8.0
748,12518031,30314,1,fixed,fixed,20160306,20160306,11.0


In [192]:
b=data_online1[mask2].groupby(['User_id'])['Action'].count().reset_index()
b.columns = ['User_id','Num_coupon_used']

In [193]:
b [:5]

,User_id,Num_coupon_used
0,209,1
1,215,2
2,387,2
3,584,2
4,696,3


In [194]:
data_online2 = pd.merge(data_online1, b, on='User_id', how='left')

In [195]:
data_online2 [:5]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Tot_coupon_got,Num_coupon_used
0,13740231,18907,2,100017492,500:50,20160513,null,6.0,NaN
1,13740231,34805,1,null,null,null,20160321,6.0,NaN
2,14336199,18907,0,null,null,null,20160618,NaN,NaN
3,14336199,18907,0,null,null,null,20160618,NaN,NaN
4,14336199,18907,0,null,null,null,20160618,NaN,NaN


# The average amount of coupon the customer got a week.

### The training set is collected from Jan. 1st 2016 to June 30th 2016.  26 weeks in total

In [196]:
def to_float (s):
    return float (s)

In [197]:
data_online3 = data_online2
data_online3['Tot_coupon_got']=data_online3['Tot_coupon_got'].map(to_float)
data_online3['Avg_coupon_got']=data_online3['Tot_coupon_got']*1/26

In [198]:
data_online3[:5]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Tot_coupon_got,Num_coupon_used,Avg_coupon_got
0,13740231,18907,2,100017492,500:50,20160513,null,6.0,NaN,0.230769
1,13740231,34805,1,null,null,null,20160321,6.0,NaN,0.230769
2,14336199,18907,0,null,null,null,20160618,NaN,NaN,NaN
3,14336199,18907,0,null,null,null,20160618,NaN,NaN,NaN
4,14336199,18907,0,null,null,null,20160618,NaN,NaN,NaN


In [199]:
data_online3.to_csv('data_online3.csv')

# (2) Offline

# Number of time this customer gets a coupon

In [200]:
data_offline[5:10]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
5,1439408,2632,null,null,0,null,20160516
6,1439408,2632,8591,20:1,0,20160516,20160613
7,1832624,3381,7610,200:20,0,20160429,null
8,2029232,3381,11951,200:20,1,20160129,null
9,2029232,450,1532,30:5,0,20160530,null


In [201]:
a=data_offline[['User_id','Coupon_id']]
mask = data_offline['Coupon_id']!='null'

In [202]:
num_coupon = a[mask].groupby(['User_id']).count().reset_index()
num_coupon.columns = ['User_id','Tot_coupon_got']
data_offline1 = pd.merge(data_offline, num_coupon, on = 'User_id',how='left')
data_offline1 [:5]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Tot_coupon_got
0,1439408,2632,null,null,0,null,20160217,5.0
1,1439408,4663,11002,150:20,1,20160528,null,5.0
2,1439408,2632,8591,20:1,0,20160217,null,5.0
3,1439408,2632,1078,20:1,0,20160319,null,5.0
4,1439408,2632,8591,20:1,0,20160613,null,5.0


# Number of time this customer uses a coupon

In [203]:
mask2 = (data_offline1['Date']!= 'null') & (data_offline1['Coupon_id']!= 'null')
b=data_offline1[mask2].groupby(['User_id'])['Coupon_id'].count().reset_index()
b.columns = ['User_id','Num_coupon_used']
data_offline2 = pd.merge(data_offline1, b, on='User_id',how='left')
data_offline2[:5]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Tot_coupon_got,Num_coupon_used
0,1439408,2632,null,null,0,null,20160217,5.0,1.0
1,1439408,4663,11002,150:20,1,20160528,null,5.0,1.0
2,1439408,2632,8591,20:1,0,20160217,null,5.0,1.0
3,1439408,2632,1078,20:1,0,20160319,null,5.0,1.0
4,1439408,2632,8591,20:1,0,20160613,null,5.0,1.0


# The average distance of the shops this customer uses a coupon.

In [204]:
null_mask = data_offline2['Distance']!='null'

def to_float(s):
    return float(s)

data_offline3 =data_offline2[null_mask]
data_offline3['Distance']=data_offline3['Distance'].map(to_float)
data_offline4 = pd.DataFrame(data_offline3.groupby('User_id')['Distance'].mean()).reset_index()
data_offline5 = pd.merge(data_offline2, data_offline4,on='User_id',how='left')

C:\Users\Jianing Cui\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [205]:
data_offline5 [:5]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance_x,Date_received,Date,Tot_coupon_got,Num_coupon_used,Distance_y
0,1439408,2632,null,null,0,null,20160217,5.0,1.0,0.142857
1,1439408,4663,11002,150:20,1,20160528,null,5.0,1.0,0.142857
2,1439408,2632,8591,20:1,0,20160217,null,5.0,1.0,0.142857
3,1439408,2632,1078,20:1,0,20160319,null,5.0,1.0,0.142857
4,1439408,2632,8591,20:1,0,20160613,null,5.0,1.0,0.142857


In [206]:
data_offline5=data_offline5.rename(columns = {'Distance_y':'Avg_Distance'})

# The average amount of coupon the customer got a week.

### The training set is collected from Jan. 1st 2016 to June 30th 2016.  26 weeks in total

In [208]:
def to_float (s):
    return float (s)
data_offline6 = data_offline5
data_offline6['Tot_coupon_got']=data_offline6['Tot_coupon_got'].map(to_float)
data_offline6['Avg_coupon_got']=data_offline6['Tot_coupon_got']*1/26

In [209]:
data_offline6 [:5]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance_x,Date_received,Date,Tot_coupon_got,Num_coupon_used,Avg_Distance,Avg_coupon_got
0,1439408,2632,null,null,0,null,20160217,5.0,1.0,0.142857,0.192308
1,1439408,4663,11002,150:20,1,20160528,null,5.0,1.0,0.142857,0.192308
2,1439408,2632,8591,20:1,0,20160217,null,5.0,1.0,0.142857,0.192308
3,1439408,2632,1078,20:1,0,20160319,null,5.0,1.0,0.142857,0.192308
4,1439408,2632,8591,20:1,0,20160613,null,5.0,1.0,0.142857,0.192308


In [210]:
data_offline6.to_csv('data_offline6.csv')

# (3) Online Vs Offline

## The relative ratio of a customer’s online and offline purchasing within those months

In [211]:
data_bothline = pd.merge(data_online, data_offline, on='User_id')
data_bothline[:5]

,User_id,Merchant_id_x,Action,Coupon_id_x,Discount_rate_x,Date_received_x,Date_x,Merchant_id_y,Coupon_id_y,Discount_rate_y,Distance,Date_received_y,Date_y
0,556864,12800,1,null,null,null,20160129,3381,1807,300:30,0,20160201,null
1,556864,52114,2,100140747,10:5,20160321,null,3381,1807,300:30,0,20160201,null
2,556864,29312,1,null,null,null,20160115,3381,1807,300:30,0,20160201,null
3,15949,27712,0,null,null,null,20160624,4043,3222,30:5,null,20160531,null
4,15949,27712,0,null,null,null,20160624,5717,8192,20:5,null,20160520,null


In [212]:
mask1 = data_bothline['Date_x']!='null'
c=data_bothline[['User_id','Date_x']]
mask2 = data_bothline['Date_y']!='null'
d=data_bothline[['User_id','Date_y']]

In [228]:
num_online_purchase = c[mask1].groupby(['User_id']).count().reset_index()
num_online_purchase.columns = ['User_id','num_online_purchase']
num_offline_purchase = c[mask2].groupby(['User_id']).count().reset_index()
num_offline_purchase.columns = ['User_id','num_offline_purchase']

In [229]:
num_bothline = pd.merge(num_online_purchase, num_offline_purchase, on='User_id',how='outer')

In [231]:
num_bothline['offline/online_ratio'] = num_bothline['num_offline_purchase']/num_bothline['num_online_purchase']

In [232]:
data_bothline2 = pd.merge(data_bothline,num_bothline,on='User_id',how='left')
data_bothline2.to_csv('data_bothline2.csv')